In [0]:
import numpy as np
import pandas as pd

from google.colab import files
uploaded = files.upload()

Saving data15smaller_4_2.csv to data15smaller_4_2.csv


In [0]:
!unzip "data16smaller_1.csv.zip"

Archive:  data18smaller_1.csv.zip
  inflating: data18smaller_1.csv     


In [0]:
data16smaller = pd.read_csv("data15smaller_3_2.csv", low_memory = False, dtype = {'subsidies_flag':"int64"})
data16smaller.head()

In [0]:
!pip install pgmpy==0.1.7
from pgmpy.estimators import HillClimbSearch, BicScore, BayesianEstimator
from pgmpy.models import BayesianModel
from sklearn.model_selection import train_test_split
from pgmpy.inference import VariableElimination

In [0]:
from collections import defaultdict
from pgmpy.inference import VariableElimination
def temp_predict(model,data):
    missing_variables = list(set(model.nodes()) - set(data.columns))
    pred_values = defaultdict(list)
    model_inference = VariableElimination(model)
    for index, data_point in data.iterrows():
        states_dict = model_inference.map_query(variables=missing_variables, evidence=data_point.astype(int).to_dict())
        for k, v in states_dict.items():
            pred_values[k].append(v)
    return pd.DataFrame(pred_values, index=data.index)

In [0]:
data16train, data16test = train_test_split(data16smaller,random_state=9,test_size=0.2)
hc = HillClimbSearch(data16train, scoring_method=BicScore(data16train))
best_model = hc.estimate()
print(best_model.edges())

[('bldgclass', 'unit_ratio'), ('bldgclass', 'subsidies_flag'), ('bldgclass', 'yearbuilt'), ('bldgclass', 'violations'), ('subsidies_flag', 'residfar'), ('residfar', 'yearbuilt'), ('yearbuilt', 'unit_ratio'), ('yearbuilt', 'violations'), ('landuse', 'residfar'), ('landuse', 'bldgclass')]


In [0]:
best_model.fit(data16train, estimator=BayesianEstimator, prior_type='K2')

for cpd in best_model.get_cpds():
    print("CPD of {variable}:".format(variable=cpd.variable))
    print(cpd)
    
print best_model.local_independencies('violations')

for thenode in best_model.nodes():
    print best_model.active_trail_nodes(thenode)

CPD of bldgclass:
+--------------+------------------------+-----------------------+------------------------+------------------------+
| landuse      | landuse(0)             | landuse(1)            | landuse(2)             | landuse(3)             |
+--------------+------------------------+-----------------------+------------------------+------------------------+
| bldgclass(0) | 0.17737789203084833    | 7.019021548396153e-05 | 0.00019708316909735908 | 0.00018076644974692697 |
+--------------+------------------------+-----------------------+------------------------+------------------------+
| bldgclass(1) | 0.8213367609254498     | 7.019021548396153e-05 | 0.00019708316909735908 | 0.00018076644974692697 |
+--------------+------------------------+-----------------------+------------------------+------------------------+
| bldgclass(2) | 0.00032133676092544985 | 0.9589387239418825    | 0.00019708316909735908 | 0.50234996384671       |
+--------------+------------------------+-------------

In [0]:
best_model.fit(data16train, estimator=BayesianEstimator, prior_type='K2')
test_x = data16test.drop(columns=['violations'])
predicted_test = temp_predict(best_model, test_x)
print "Out of sample:",(data16test.loc[:,'violations']==predicted_test['violations']).mean()

Out of sample: 0.6324076724878328


In [0]:
train_x = data16train.drop(columns=['violations'])
predicted_train = temp_predict(best_model, train_x)
print "In of sample:",(data16train.loc[:,'violations']==predicted_train['violations']).mean()

In of sample: 0.6352671701084428


In [0]:
data16smaller = pd.read_csv("data15smaller_3_2.csv", low_memory = False, dtype = {'subsidies_flag':"int64"})
data16smaller.head()

In [0]:
data16train, data16test = train_test_split(data16smaller,random_state=9,test_size=0.2)
hc = HillClimbSearch(data16train, scoring_method=BicScore(data16train))
best_model = hc.estimate()
print(best_model.edges())

[('bldgclass', 'unit_ratio'), ('bldgclass', 'subsidies_flag'), ('bldgclass', 'yearbuilt'), ('bldgclass', 'vio_rt'), ('subsidies_flag', 'residfar'), ('residfar', 'yearbuilt'), ('yearbuilt', 'unit_ratio'), ('yearbuilt', 'vio_rt'), ('landuse', 'residfar'), ('landuse', 'bldgclass')]


In [0]:


best_model.fit(data16train, estimator=BayesianEstimator, prior_type='K2')

train_x = data16train.drop(columns=['vio_rt'])
predicted_train = temp_predict(best_model, train_x)
print "In of sample:",(data16train.loc[:,'vio_rt']==predicted_train['vio_rt']).mean()

test_x = data16test.drop(columns=['vio_rt'])
predicted_test = temp_predict(best_model, test_x)
print "Out of sample:",(data16test.loc[:,'vio_rt']==predicted_test['vio_rt']).mean()

In of sample: 0.3536022332772628
Out of sample: 0.3509876896650444


In [0]:

for cpd in best_model.get_cpds():
    print("CPD of {variable}:".format(variable=cpd.variable))
    print(cpd)
    
print best_model.local_independencies('vio_rt')

for thenode in best_model.nodes():
    print best_model.active_trail_nodes(thenode)

CPD of bldgclass:
+--------------+------------------------+-----------------------+------------------------+------------------------+
| landuse      | landuse(0)             | landuse(1)            | landuse(2)             | landuse(3)             |
+--------------+------------------------+-----------------------+------------------------+------------------------+
| bldgclass(0) | 0.17737789203084833    | 7.019021548396153e-05 | 0.00019708316909735908 | 0.00018076644974692697 |
+--------------+------------------------+-----------------------+------------------------+------------------------+
| bldgclass(1) | 0.8213367609254498     | 7.019021548396153e-05 | 0.00019708316909735908 | 0.00018076644974692697 |
+--------------+------------------------+-----------------------+------------------------+------------------------+
| bldgclass(2) | 0.00032133676092544985 | 0.9589387239418825    | 0.00019708316909735908 | 0.50234996384671       |
+--------------+------------------------+-------------